In [14]:
import ast
import os
from datetime import date, datetime, timedelta

import geopandas as gpd
import pandas as pd
from dotenv import load_dotenv
from shapely.geometry import Point
from sqlalchemy import create_engine

In [15]:
def in_boundary(city_geo_data, lat, lon):
    loc_p = Point(lon, lat)
    return city_geo_data.contains(loc_p)

In [16]:
folder = r"C:\Users\add41\Documents\Data_Engineer\Project\example_data"
file = "COUNTY_MOI_1140318.gml"
geo_path = os.path.join(folder, file)
gdf = gpd.read_file(geo_path)

bound_dict = {row["名稱"]:row.geometry for _, row in gdf.iterrows()}

In [17]:
# 設定資料夾、城市和類別列表
folder = r"C:\Users\add41\Documents\Data_Engineer\Project\example_data\gmap_full_search"
city_list = ["TPE", "TYU", "TCH", "TNA", "KSH"]
type_list = ["salon", "restaurant", "supplies"]
city_dict = {
    "TPE":"新北市",
    "TYU":"桃園市",
    "TCH":"臺中市",
    "TNA":"臺南市",
    "KSH":"高雄市"
}

# 先按照類別迴圈
for type_ in type_list:
    columns = ['name', 'place_id', 'buss_status', "geometry", 'update_time']
    df_main = pd.DataFrame(columns=columns)

    # 依序將城市檔案讀入
    for city in city_list:
        file = f"gmap_{type_}_{city}.csv"
        path = os.path.join(folder, file)
        df_city = pd.read_csv(path)

        # 只保留正常營業的地標
        mask1 = df_city["buss_status"] == "OPERATIONAL"
        df_city = df_city[mask1]

        # 根據place id去除重複
        df_city = df_city.drop_duplicates(subset=["place_id"], keep="first")

        # 去除無經緯度的資料
        df_city.dropna(subset="geometry", inplace=True)
        print(f"完成{city}資料處理，共{len(df_city)}筆資料")

        # 檢查店家是否在六都邊界範圍內
        city_ch_name = city_dict[city]
        city_geo_data = bound_dict[city_ch_name]

        boundary_list = []
        df_city["geometry"] = df_city["geometry"].apply(ast.literal_eval)
        for index, row in df_city.iterrows():
            lat = row["geometry"].get("lat", None)
            lon = row["geometry"].get("lng", None)
            if lat == None or lon == None:
                boundary_list.append(False)
            else:
                boundary_list.append(in_boundary(city_geo_data, lat, lon))

        df_city['in_boundary'] = boundary_list

        # 只保留在邊界範圍內的
        mask2 = (df_city['in_boundary'] == True)
        df_city = df_city[mask2]

        # 跟主表合併
        df_main = pd.concat([df_main, df_city], ignore_index=True)

    # 合併完成再去重複一次
    df_main = df_main.drop_duplicates(subset=["place_id"], keep="first")

    # 去除經緯度欄位
    df_main.drop(columns="geometry", axis=1, inplace=True)

    # 新增更新日期
    today = date.today().strftime('%Y/%m/%d')
    df_main["update_date"] = today

    # 存檔至地端
    combine_file = f"{type_}_place_id.csv"
    combine_path = os.path.join(folder, combine_file)
    df_main.to_csv(combine_path, index=False)
    print(f"已完成{type_}地端存檔")

    # 更新至資料庫
    username = os.getenv("MYSQL_USERNAME")
    password = os.getenv("MYSQL_PASSWORD")
    target_ip = os.getenv("MYSQL_IP")
    target_port = int(os.getenv("MYSQL_PORTT"))
    db_name = os.getenv("MYSQL_DB_NAME")

    engine = create_engine(f"mysql+pymysql://{username}:{password}@{target_ip}:{target_port}/{db_name}")

    df_main.to_sql(name=f"{type_}_place_id", con=engine, if_exists="replace", index=False)
    print(f"已完成{type_}資料庫存檔")

完成TPE資料處理，共697筆資料
完成TYU資料處理，共447筆資料
完成TCH資料處理，共457筆資料
完成TNA資料處理，共339筆資料
完成KSH資料處理，共391筆資料
已完成salon地端存檔
已完成salon資料庫存檔
完成TPE資料處理，共975筆資料
完成TYU資料處理，共608筆資料
完成TCH資料處理，共647筆資料
完成TNA資料處理，共588筆資料
完成KSH資料處理，共455筆資料
已完成restaurant地端存檔
已完成restaurant資料庫存檔
完成TPE資料處理，共568筆資料
完成TYU資料處理，共348筆資料
完成TCH資料處理，共426筆資料
完成TNA資料處理，共381筆資料
完成KSH資料處理，共359筆資料
已完成supplies地端存檔
已完成supplies資料庫存檔
